<a href="https://colab.research.google.com/github/UmarKhattab09/WebScarping-LLM-FineTuning/blob/main/WebScarping_for_LLM_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import json

In [ ]:
urldict = {
    'politics':f"https://www.npr.org/get/1014/render/partial/next?start={1}&count={24}",
    'science' : f"https://www.npr.org/get/1007/render/partial/next?start={1}&count={24}",
    'health'  : f"https://www.npr.org/get/1128/render/partial/next?start={1}&count={24}",
    'business': f"https://www.npr.org/get/1006/render/partial/next?start={1}&count={24}",
    'climate' : f"https://www.npr.org/get/1167/render/partial/next?start={1}&count={24}" #
}

In [ ]:
def url_extractor(url):
  response=requests.get(url)
  status = response.status_code
  #print(f"Status:{status}")
  html_content= response.text
  soup = BeautifulSoup(html_content,'html.parser')
  return soup

politics= url_extractor(urldict['politics'])
science= url_extractor(urldict['science'])
health= url_extractor(urldict['health'])
business= url_extractor(urldict['business'])
climate= url_extractor(urldict['climate'])


In [ ]:
def extractinglinks(url):
  data=[]
  for article in url.find_all('article'):
    anchortag=article.find('a')
    try:
      articlelink=anchortag['href']
      if articlelink.startswith("https://www.npr.org"):
        data.append(articlelink)
    except:
      f"no href"
  return data
politicslinks=extractinglinks(politics)
sciencelinks=extractinglinks(science)
healthlinks=extractinglinks(health)
businesslinks=extractinglinks(business)
climatelinks=extractinglinks(climate)

politicslinks[23]

# data=[]
# for article in politics.find_all('article'):
#   anchortag=article.find('a')
#   try:
#     articlelink=anchortag['href']
#     if articlelink.startswith("https://www.npr.org"):
#       data.append(articlelink)
#   except:
#     f"no href"
# print("___________________________")


#   articleur=anchortag['href']
#   #print(articleur)
#   if articleur.startswith('https://www.npr.org'):


'https://www.npr.org/2025/07/11/nx-s1-5459955/social-security-megabill-trump-tax-cuts'

In [ ]:
# data=[]
# for article in climate.find_all('article'):
#   anchortag=article.find('a')
#   try:
#     articlelink=anchortag['href']
#     if articlelink.startswith("https://www.npr.org"):
#       data.append(articlelink)
#   except:
#     f"no href"
# print("___________________________")


# #   articleur=anchortag['href']
# #   #print(articleur)
# #   if articleur.startswith('https://www.npr.org'):
# #     data.append(articleur)
# print(data)
# print(len(data))

In [ ]:
def storyextractor(links):
  separator=' '
  data =[]
  for i in range(len(links)):
    article = url_extractor(links[i])
    storydiv=article.find("div",id="storytext")
    try:
      text=storydiv.find_all('p')
    except:
      f"no p"
    for i in range(len(text)):
      try:
        temp= text[i].get_text(separator= ' ', strip = True)
      except:
        f"no temp"
      text[i] = temp
    finaltxt= separator.join(text)
    data.append(finaltxt)
  return data


politicsnews = storyextractor(politicslinks)
sciencenews = storyextractor(sciencelinks)
healthnews = storyextractor(healthlinks)
businessnews = storyextractor(businesslinks)
climatenews = storyextractor(climatelinks)



In [ ]:
import pandas as pd
def createDataFrame(Name:str,data:list):
  df = pd.DataFrame({
      "NewsType": [Name] * len(data),
      "News": data
  })
  print(f"The size of News for {Name} is {len(data)}")
  return df
politicsdf = createDataFrame("Politics",politicsnews)
sciencedf = createDataFrame("Science",sciencenews)
healthdf = createDataFrame("Health",healthnews)
businessdf = createDataFrame("Business",businessnews)
climatedf = createDataFrame("Climate",climatenews)


The size of News for Politics is 24
The size of News for Science is 24
The size of News for Health is 24
The size of News for Business is 24
The size of News for Climate is 24


In [ ]:
combined_df_rows = pd.concat([politicsdf, sciencedf,healthdf,businessdf,climatedf])
combined_df_rows.to_csv("Dataset.csv")

#### Fine Tuning LLM
1. Define Parameter
2. Clean Dataset
3. Wrangling Dataset

>> Label Encoder

>> TrainTestSplit

>> Convert into Hugging Face

>> Tokenizer

4. Initalize Model
5. Train Model


In [ ]:
import pandas as pd
import huggingface_hub

model_name="meta-llama/Llama-3.2-1B"


In [ ]:
text_column_name='article'
df = pd.read_csv("Dataset.csv")

In [ ]:
df['NewsType'].nunique()

5

In [ ]:
from google.colab import userdata
hf_token=userdata.get("hf_token")
huggingface_hub.login(hf_token)

In [ ]:
import re
class Cleaner():
  def __init__(self):
    pass

  def remove_hml_tags(self,text):
    clean_text=BeautifulSoup(text,'lxml').text
    return clean_text

  def remove_double_spaces(self,text):
    clean_text=re.sub(' +',' ',text)
    return clean_text
  def clean(self,text):
    clean_text=self.remove_hml_tags(text)
    clean_text=self.remove_double_spaces(clean_text)
    return clean_text



In [ ]:
from bs4 import BeautifulSoup
import re
cleaner=Cleaner()
df['text_cleaned']=df['News'].apply(cleaner.clean)

In [ ]:
df.head()

,Unnamed: 0,NewsType,News,text_cleaned
0,0,Politics,President Trump in profile at MetLife Stadium...,President Trump in profile at MetLife Stadium ...
1,1,Politics,"GOP Rep. Hal Rogers of Kentucky, left, poses d...","GOP Rep. Hal Rogers of Kentucky, left, poses d..."
2,2,Politics,A sign shows the location of an immigration co...,A sign shows the location of an immigration co...
3,3,Politics,"Mike Waltz, former US national security advise...","Mike Waltz, former US national security advise..."
4,4,Politics,Workers on scaffolding repaint the NASA logo n...,Workers on scaffolding repaint the NASA logo n...


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


In [ ]:
le.fit(df['NewsType'].tolist())
df['label']=le.transform(df['NewsType'].tolist())

In [ ]:
df.sample()

,Unnamed: 0,NewsType,News,text_cleaned,label
50,2,Health,"Megan and Stephen Alger, photographed at home ...","Megan and Stephen Alger, photographed at home ...",2


In [ ]:
from sklearn.model_selection import train_test_split
train_df,test_df=train_test_split(df,test_size=0.2,random_state=42)

In [ ]:
df_train=train_df[['text_cleaned','label']]
df_test=test_df[['text_cleaned','label']]

#### Convert To HuggingFACE Dataset

In [ ]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
dataset_train=Dataset.from_pandas(df_train)
dataset_test=Dataset.from_pandas(df_test)

In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name)

## Removing output to apply on github

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id=tokenizer.eos_token_id
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/tokenizer.json')

In [ ]:
def preprocessing(examples):
  return tokenizer(examples['text_cleaned'],truncation=True)

tokenized_train=dataset_train.map(preprocessing,batched=True)
tokenized_test=dataset_test.map(preprocessing,batched=True)

In [ ]:
  #Intialize Model
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=5)

In [ ]:
model.config.pad_token_id=model.config.eos_token_id

In [ ]:
numberoflayers=0
for param in model.base_model.parameters():
  numberoflayers+=1
print(numberoflayers)

146


In [ ]:
layersnumber=0
for param in model.base_model.parameters():
  if layersnumber>numberoflayers-25:
    break
  layersnumber+=1
  param.requires_grad=False


In [ ]:
#TrainModel
from transformers import TrainingArguments,Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
metric=evaluate.load("accuracy")
def compute_metrics(eval_pred):
  logits,labels=eval_pred
  predictions=np.argmax(logits,axis=-1)
  return metric.compute(predictions=predictions,references=labels)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id  # match model to tokenizer

In [ ]:
print("Tokenizer pad_token:", tokenizer.pad_token)
print("Tokenizer pad_token_id:", tokenizer.pad_token_id)
print("Model config pad_token_id:", model.config.pad_token_id)

Tokenizer pad_token: <|end_of_text|>
Tokenizer pad_token_id: 128001
Model config pad_token_id: 128001


In [ ]:
training_args=TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,

    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    fp16=True,
    report_to="none",

      save_steps=2000
)
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-42-2540101056.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [ ]:
trainer.train()

In [ ]:
from sklearn.metrics import classification_report
preds = trainer.predict(tokenized_test)
preds = np.argmax(preds.predictions, axis=-1)
GT= df_test['label'].tolist()
print(classification_report(GT, preds))

              precision    recall  f1-score   support

           0       0.67      0.50      0.57         4
           1       0.00      0.00      0.00         3
           2       0.30      0.60      0.40         5
           3       0.40      0.40      0.40         5
           4       0.50      0.14      0.22         7

    accuracy                           0.33        24
   macro avg       0.37      0.33      0.32        24
weighted avg       0.40      0.33      0.33        24



In [ ]:
model.config.id2label= {i:label for i,label in enumerate(le.classes_)}
model.config.idlabel2= {i:label for i,label in enumerate(le.classes_)}

In [ ]:
trainer.save_model("./news_classifier_model")
tokenizer.save_pretrained("./news_classifer_model")

('./news_classifer_model/tokenizer_config.json',
 './news_classifer_model/special_tokens_map.json',
 './news_classifer_model/tokenizer.json')

In [ ]:
from transformers import pipeline
classifier=pipeline("text-classification",model="./news_classifier_model",tokenizer="./news_classifier_model")
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)

Device set to use cuda:0
Device set to use cuda:0


In [ ]:
text = """
ndependence Day is approaching!

Imagine if, in a few days, someone has procured illegal fireworks from a couple of states over. Are you:

first in line to light them
content to watch while others set them off
going to find a fire extinguisher — just in case — while loudly condemning the activity?
Ken Carter, a psychologist at Oxford College of Emory University, says everyone has a different level of sensation-seeking. This episode, we get into the factors at play, like people's brain chemistry, when deciding whether or not to do an activity, like setting off fireworks.

For low and average sensation-seekers, very thrilling activities like large, self-run fireworks displays can cause their bodies to produce a lot of cortisol, a stress hormone.

On the other hand, high sensation-seekers, Carter says, "don't tend to produce that much cortisol when they're in those highly chaotic experiences. So when they're seeing those fireworks, they actually produce higher amounts of another chemical called dopamine, which is a neurotransmitter or a chemical messenger that's involved in pleasure."

Carter has developed a 40-point self-assessment survey for people to figure out how much of a sensation-seeker they are. The survey can be found in his book, Buzz!
"""

In [ ]:
result = clf.predict(text)
print(result)

[{'label': np.str_('Health'), 'score': 0.9915760159492493}]
